In [39]:
import io, os
import numpy as np
from google.cloud import vision
from PIL import ImageTk, Image, ImageDraw
import pandas as pd
import sqlite3
import json

In [40]:
# create client
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="static/cloud_credentials_steel-operative-337913-b3f7be97f534.json"
client = vision.ImageAnnotatorClient()

In [41]:
img_path = "static/images/paintings/"
paintings = os.listdir(img_path)

In [42]:
con = sqlite3.connect('rijksstudio.db')
cur = con.cursor()

In [43]:
# create objects table
cur.execute("""CREATE TABLE IF NOT EXISTS Objects
               (object_id TEXT PRIMARY KEY,
                painting_id TEXT,
                object_name TEXT,
                confidence INTEGER,
                min_x INTEGER, 
                min_y INTEGER,
                max_x INTEGER, 
                max_y INTEGER,
                source TEXT
                )""")

# Save changes
con.commit()

In [44]:
def get_object_data(obj, painting, dims):
    normalized_vertices = obj.bounding_poly.normalized_vertices
    
    q = cur.execute("""INSERT OR REPLACE INTO Objects 
                       VALUES (?,?,?,?,?,?,?,?,?);""", 
                       (create_obj_id(painting,obj.name), 
                        painting, 
                        obj.name, 
                        obj.score, 
                        int(np.round(normalized_vertices[0].x * dims[0])), 
                        int(np.round(normalized_vertices[0].y * dims[1])), 
                        int(np.round(normalized_vertices[2].x * dims[0])), 
                        int(np.round(normalized_vertices[2].y * dims[1])), 
                        "GOOGLE_VISION"))

    con.commit()

In [45]:
# turns a combination of painting and object name to an id
def create_obj_id(painting,obj_name):
    obj_id = ''.join([str(ord(char)-96) for char in obj_name])    # convert chars to numbers
    return (painting.split(".")[0] + obj_id) + "G"    # + G for Google Vision

In [46]:
for painting in paintings:
    print("Reading "+painting+" ...")
    # Loads the image into memory
    with io.open(os.path.join(img_path, painting), 'rb') as image_file:
        content = image_file.read()
        dims = Image.open(image_file).size
        
    # call API
    image = vision.Image(content=content)
    response = client.object_localization(image=image)
    localized_object_annotations = response.localized_object_annotations
    
    objects = [obj.name for obj in localized_object_annotations]
    unique_objs = [x for x in objects if objects.count(x) ==1]
    
    # skip if no unique objects
    if len(unique_objs)<1:
        continue
        
    # add every object data to db
    for obj in localized_object_annotations:
        normalized_vertices = obj.bounding_poly.normalized_vertices
        
        if obj.name in unique_objs:
            objects.append(obj)
            get_object_data(obj, painting, dims)
    print("objects "+str(unique_objs)+" added to db")

Reading RP-F-2004-71.png ...
objects ['Pitcher'] added to db
Reading RP-F-2014-7-1-3.png ...
objects ['Vase', 'Tableware'] added to db
Reading RP-P-H-1086.png ...
objects ['Glasses'] added to db
Reading RP-T-1881-A-107.png ...
objects ['Crab', 'Animal'] added to db
Reading RP-T-1951-200.png ...
objects ['Food', 'Fruit', 'Insect'] added to db
Reading RP-T-1951-203.png ...
objects ['Packaged goods', 'Fruit', 'Food'] added to db
Reading SK-A-1107.png ...
objects ['Packaged goods', 'Lemon', 'Tableware', 'Moths and butterflies', 'Food'] added to db
Reading SK-A-113.png ...
objects ['Shoe', 'Luggage & bags', 'Clock'] added to db
Reading SK-A-1130.png ...
objects ['Picture frame'] added to db
Reading SK-A-129.png ...
Reading SK-A-137.png ...
objects ['Lighting'] added to db
Reading SK-A-1451.png ...
Reading SK-A-1595.png ...
objects ['Racket', 'Luggage & bags', 'Picture frame', 'Footwear'] added to db
Reading SK-A-1627.png ...
Reading SK-A-1649.png ...
objects ['Dress'] added to db
Reading SK

In [47]:
# simulate clicks to prompt the painting to users with default user id 999999
def simulate_clicks():
    # retrieve all objects from db
    q = cur.execute("""SELECT * FROM Objects""")
    cols = [column[0] for column in q.description]
    df = pd.DataFrame.from_records(data = q.fetchall(), columns = cols)
    
    simulations = 250
    
    # calculate center of bouding box
    df["bounding_center"] = df.apply(lambda row : ((row["max_x"]-row["min_x"])/2,(row["max_y"]-row["min_y"])/2,), axis = 1)

    # generate clicks per object 
    for i in range(len(df)):
        x = np.random.normal(df.loc[3,"bounding_center"][0], 0.02, simulations)
        y = np.random.normal(df.loc[3,"bounding_center"][1], 0.02, simulations)

        # add simulations to db
        for j in range(simulations):
            # Store click in database
            cur.execute(f"""INSERT INTO Clicks
                    VALUES ("{df.loc[i,"painting_id"]}", 
                            "{df.loc[i,"object_name"]}", 
                            "{999999}", 
                            {x[j]}, 
                            {y[j]},
                            100)""")
        print(df.loc[i,"painting_id"] + " done")

con.commit()

In [48]:
simulate_clicks()

RP-F-2004-71.png done
RP-F-2014-7-1-3.png done
RP-F-2014-7-1-3.png done
RP-P-H-1086.png done
RP-T-1881-A-107.png done
RP-T-1881-A-107.png done
RP-T-1951-200.png done
RP-T-1951-200.png done
RP-T-1951-200.png done
RP-T-1951-203.png done
RP-T-1951-203.png done
RP-T-1951-203.png done
SK-A-1107.png done
SK-A-1107.png done
SK-A-1107.png done
SK-A-1107.png done
SK-A-1107.png done
SK-A-113.png done
SK-A-113.png done
SK-A-113.png done
SK-A-1130.png done
SK-A-137.png done
SK-A-1595.png done
SK-A-1595.png done
SK-A-1595.png done
SK-A-1595.png done
SK-A-1649.png done
SK-A-1751.png done
SK-A-180.png done
SK-A-180.png done
SK-A-180.png done
SK-A-1857.png done
SK-A-199.png done
SK-A-2150.png done
SK-A-2152.png done
SK-A-2152.png done
SK-A-2180.png done
SK-A-2180.png done
SK-A-2251.png done
SK-A-2251.png done
SK-A-2344.png done
SK-A-2344.png done
SK-A-2362.png done
SK-A-2366.png done
SK-A-2388.png done
SK-A-2388.png done
SK-A-2452.png done
SK-A-2565.png done
SK-A-263.png done
SK-A-263.png done
SK-A-26

In [49]:
con.close()